In [1]:
from pyspark import *
#from pyspark.sql.functions import *

# Exploratory data analysis (EDA)

In [3]:
# Carrega dataset do desafio
#display(dbutils.fs.ls("dbfs:/FileStore/tables/"))
df_desafio = spark.read.format("csv").options(header='true').load("/FileStore/tables/*.csv")

In [4]:
# Elimina na's e duplicados do df
df_desafio_v2 = df_desafio.dropna(how='any').dropDuplicates()

# Ajusta tipo de colunas
df_desafio_v2 = df_desafio.selectExpr(
  'cast(time as timestamp) time',
  'ca',
  'unit',
  'scp',
  'station',
  'linename',
  'division',
  'desc',
  'cast(entries as int) entries',
  'cast(exits as int) exits'
)

# Features para visão temporal
df_desafio_v2 = df_desafio_v2.withColumn(
  "dt_year",
  year(col("time"))
).withColumn(
  "dt_month",
  month(col("time"))
).withColumn(
  "dt_day",
  dayofmonth(col("time"))
).withColumn(
  "dt_dayofy",
  dayofyear(col("time"))
).withColumn(
  "dt_hour",
  hour(col("time"))
).withColumn(
  "dt_min",
  minute(col("time"))
).withColumn(
  "dt_week_no",
  weekofyear(col("time"))
).withColumn(
  "dt_int",
  unix_timestamp(col("time"))
)

In [5]:
# Check: 79609191 / 79130015 / 79045675
count_desafio = df_desafio.count()
count_desafio_na = df_desafio.dropna(how='any').count()
count_desafio_final = df_desafio.dropna(how='any').dropDuplicates().count()

df_amostras = sc.parallelize([
  ('antes',count_desafio,0,0,0),
  ('depois',0,count_desafio_final,count_desafio-count_desafio_na,count_desafio_na-count_desafio_final)
]).toDF(['AMOSTRAS','TOTAL','UNICO','NA','DUPLICADO'])

display(df_amostras)

In [6]:
df_amostras_v2 = sc.parallelize([
  ('',count_desafio-count_desafio_na,count_desafio_na-count_desafio_final)
]).toDF(['AMOSTRAS','NA','DUPLICADO'])

display(df_amostras_v2)

In [7]:
# Método para dummys
pivot_cols = ['ca', 'unit', 'scp', 'station', 'linename', 'division', 'desc']
keys = df_desafio_v2.columns
before = df_desafio_v2

def join_all(dfs,keys):
    if len(dfs) > 1:
        return dfs[0].join(join_all(dfs[1:],keys), on = keys, how = 'inner')
    else:
        return dfs[0]

dfs = []
combined = []
for pivot_col in pivot_cols:
    pivotDF = before.groupBy(keys).pivot(pivot_col).count()
    new_names = pivotDF.columns[:len(keys)] +  ["e_{0}_{1}".format(pivot_col, c) for c in pivotDF.columns[len(keys):]]        
    df = pivotDF.toDF(*new_names).fillna(0)    
    combined.append(df)

df_desafio_v3 = join_all(combined,keys)

#contagem = df_desafio_v3.columns
#len(contagem) ==> 2286 features

In [8]:
#-7254
#Out[26]: 79052929

df_desafio_v3.count()

In [9]:
#display(df_desafio_v2)

In [10]:
#display(df_desafio_v2)